# 🔧 YOLOv8 Road Defect Detection with GPS (Google Drive Access)
This notebook processes video frames to detect road anomalies using a YOLOv8 model and extract GPS coordinates along with human-readable addresses.

In [ ]:
# 📦 Install Required Packages
!pip install ultralytics geopy opencv-python-headless pandas

In [ ]:
# 🔗 Mount Google Drive to access your model and video
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ✅ Set the path to your YOLOv8 model and video in Google Drive
model_path = "/content/drive/MyDrive/road_detection/best.pt"
video_path = "/content/drive/MyDrive/road_detection/road_video.mp4"

In [ ]:
# 🚦 Process Video and Detect Defects
import cv2
import os
import csv
from geopy.geocoders import Nominatim
from ultralytics import YOLO
from collections import defaultdict

model = YOLO(model_path)
names = model.names

output_folder = 'yolo_gps_output'
os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))

geolocator = Nominatim(user_agent="geoapi")

csv_path = os.path.join(output_folder, 'detection_output.csv')
csv_file = open(csv_path, 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["Frame", "Latitude", "Longitude", "Location", "Defect Category"])

frame_count = 0
while True:
    success, frame = cap.read()
    if not success:
        break

    if frame_count % 10 == 0:
        results = model(frame)[0]
        defects = list(set([names[int(cls)] for cls in results.boxes.cls.cpu().numpy()]))

        if defects:
            # Replace with dynamic GPS extraction if needed
            lat, lon = 19.098404, 74.690346
            try:
                location = geolocator.reverse((lat, lon), timeout=10).address
            except:
                location = "Location not found"

            csv_writer.writerow([frame_count, lat, lon, location, ", ".join(defects)])

    frame_count += 1

cap.release()
csv_file.close()
print("✅ Processing complete. Output saved to:", csv_path)